# Imports, api, datasets and functions to use Naswot metric

In [ ]:
!pip install nats_bench
!wget "https://www.dropbox.com/sh/ceeo70u1buow681/AADxyCvBAnE6mMjp7JOoo0LVa/NATS-tss-v1_0-3ffb9-simple.tar"
!tar -xf "NATS-tss-v1_0-3ffb9-simple.tar"


!pip install xautodl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-06-26 17:22:53--  https://www.dropbox.com/sh/ceeo70u1buow681/AADxyCvBAnE6mMjp7JOoo0LVa/NATS-tss-v1_0-3ffb9-simple.tar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/ceeo70u1buow681/AADxyCvBAnE6mMjp7JOoo0LVa/NATS-tss-v1_0-3ffb9-simple.tar [following]
--2022-06-26 17:22:53--  https://www.dropbox.com/sh/raw/ceeo70u1buow681/AADxyCvBAnE6mMjp7JOoo0LVa/NATS-tss-v1_0-3ffb9-simple.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc876039a48058814224ef39a483.dl.dropboxusercontent.com/cd/0/inline/Bn_5Qaagz95omuh-QbflumgrzuMcieR_pfo7cUOavPdgO9smQJ8hlthSNsCbM56dycm2ADoKdvPNqCn_T1TgaOZE8aH2uv7CMegUakJ6R5okXCVzX3Ytzdazk8Sz

In [ ]:
!wget 'https://www.dropbox.com/s/o2fg17ipz57nru1/?dl=1' -O ImageNet16.tar.gz
!tar -xf "ImageNet16.tar.gz"

--2022-06-10 08:43:16--  https://www.dropbox.com/s/o2fg17ipz57nru1/?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/o2fg17ipz57nru1 [following]
--2022-06-10 08:43:16--  https://www.dropbox.com/s/dl/o2fg17ipz57nru1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1ad5f1e91ecd2f608ef47c408b.dl-eu.dropboxusercontent.com/cd/0/get/Bm5pV_v6GC_qCygUjfHGcX1s_CchUV8zC0F9wtkz2d6XCw4lihbF4VMjm7A6S3iQADs8cevJigob4n06BdEsQrlLfFkgaGIHKhrHkkuYRmijbvDdaHJI1vyIVhf-Jad3sXOw0npQJdRrt1wSUOY9BkT_EKEtgHbpdIQFYGyc1Qv-PQ7y216Zl94OAPXDhTdpNKc/file?dl=1# [following]
--2022-06-10 08:43:17--  https://uc1ad5f1e91ecd2f608ef47c408b.dl-eu.dropboxusercontent.com/cd/0/get/Bm5pV_v6GC_qCygUjfHGcX1s_CchUV8zC0F9wtkz2d6XCw4lihbF4VMjm7A6S3iQADs8cevJig

In [ ]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
from xautodl.datasets.DownsampledImageNet import ImageNet16
from torchvision import transforms
import numpy as np
import torch
import operator
from numba import cuda
from nats_bench import create
from xautodl.models import get_cell_based_tiny_net
import random
import pandas as pd
import os
from scipy import stats
import time
import operator
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Create the API instance for the size search space in NATS
api = create("/content/NATS-tss-v1_0-3ffb9-simple/", 'tss', fast_mode=True, verbose=False)

#tranforms to tensor the input data, no data augmentation needed
transform = transforms.Compose([
    transforms.ToTensor()
])

#funtion to retrieve the train loader
def get_data(dataset, batch_size, pin_memory=True):
  transform = transforms.Compose([
    transforms.ToTensor() ])
  if dataset == 'cifar10':
    train_data = dset.CIFAR10 ("/content/Cifar10", train=True ,transform = transform, download=True)

  elif dataset == 'cifar100':
    train_data = dset.CIFAR100("/content/Cifar100", train=True ,transform = transform, download=True)
    
  elif dataset == 'ImageNet16-120':
    train_data = ImageNet16("/content/ImageNet16", True, transform = transform)

  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=pin_memory)
  
  return train_loader


In [ ]:
#log of kernel function
def logdet(K):
    s, ld = np.linalg.slogdet(K)
    return ld

#weight initialization function
def init(m):
    if isinstance(m, (torch.nn.Conv2d, torch.nn.Linear)):
        torch.nn.init.xavier_normal_(m.weight)

# function to compute Hamming Distance, using cuda jit library
# A is the binary codification of the current Relu output
# C is the hamming distance (result of this function) 
@cuda.jit
def constructK(A,C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += operator.xor(bool(A[i, k]),bool(A[j, k]))
        C[i, j] = tmp

In [ ]:
# hook registered on forward in each Relu instance of the network
# here we binarize the output of Relu, setup the parameters to parallelize with cuda.jit
# construct the kernel matrix
def counting_forward_hook(module, inp, out):
  if isinstance(inp, tuple):
    inp = inp[0]
  inp = inp.view(inp.size(0), -1)
  inp = (inp > 0).float()

  inp = cuda.to_device(inp)
  temp = cuda.device_array((inp.shape[0], inp.shape[0]))

  threadsperblock = (8,8)
  blockspergrid = (16,16)

# Start the kernel 
  constructK[blockspergrid, threadsperblock](inp, temp)

# Copy the result back to the host
  global Ktemp, device
  temp = torch.tensor(temp.copy_to_host()).to(device)
  temp = inp.shape[1]-temp
  Ktemp = Ktemp + temp


In [ ]:
# dictionary that registers isomorphisms, useful to compute score only in the unique models
networks = {}
ids = []

for i in range(len(api)):
  # we use get_unique_str() that return a string unique for every isomorphism
  net = api.get_unique_str(i)
  if net not in networks.keys():
    networks[net] = [i]
    ids.append(i)
  else :
    networks[net].append(i)

nets = {}
for k,v in networks.items():
  if len(v) > 1:
    nets[v[0]] = v[1:]
  else:
    nets[v[0]] = []

del networks

ids = np.array(ids)

#Step 2a
Random search picking the model with highest Naswot metric, considering different sizes

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
datasetNames = ['cifar10','cifaar100','ImageNet16-120']

# number of iterations 
runs_naswot = 10
# we consider different sizes of the random sample
sample_sizes = [50,100,200,300]

batch_size = 128

for dataset in datasetNames:  
  for sample_size in sample_sizes:
    for run in range(runs_naswot):

      train_loader = get_data(dataset, batch_size)
      
      # list to trace the scores for the selected networks
      scores = []
      
      nets = np.random.randint(0,len(api),size=sample_size)

      start = time.time()

      for i in nets:

        config = api.get_net_config(i, dataset)
        network = get_cell_based_tiny_net(config)
        network.apply(init)

        for name, module in network.named_modules():
          if (isinstance(module, torch.nn.modules.activation.ReLU)):
            module.register_forward_hook(counting_forward_hook)

        network = network.to(device)

        Ktemp = torch.tensor(np.zeros((batch_size,batch_size))).to(device)
        data_iterator = iter(train_loader)
        x, target = next(data_iterator)
        x, target = x.to(device), target.to(device)
        network(x)
        score = logdet(Ktemp.cpu().detach().numpy(), target)
        
        del Ktemp, network, data_iterator
        
        scores.append(score)

      #select the best scoring function
      ind_best = np.argmax(scores)
      acc = api.get_more_info(int(nets[ind_best]),dataset,is_random=False,hp=200)['test-accuracy']
      tempo = time.time()-start
      csv_dict = {'Dataset': dataset, 'SampleSize' : sample_size,'Network': nets[ind_best], 'Metric': scores[ind_best], 'Accuracy': acc, 'Time': tempo}
      result = pd.DataFrame([csv_dict])
      result.to_csv('naswot_score.csv', mode='a', index=False, header=False )


In [ ]:
df = pd.read_csv("/content/naswot_score.csv",names = ['Dataset','size','Network','Metric','Accuracy','Time'], header = None, index_col=['Dataset','Network'])

metric = df.groupby(by = ['Dataset','size'])['Metric'].describe()[['mean','std']]
accuracy = df.groupby(by = ['Dataset','size'])['Accuracy'].describe()[['mean','std']]
time = df.groupby(by = ['Dataset','size'])['Time'].describe()[['mean','std']]

In [ ]:
data = pd.concat([metric,accuracy,time],axis=1)
data.to_csv('stats_naswot.csv')

In [ ]:
data.columns = ['metric_mean','metirc_std','acc_mean','acc_std','time_mean','time_std']

#Step 2b
Random search picking the model with highest accuracy on testSet after 12 epoches of training, considering different sizes

In [ ]:
datasetNames = ['cifar10','cifaar100','ImageNet16-120']

runs_naswot = 10
sample_sizes = [50,100,200,300]

for dataset in datasetNames:  
  for sample_size in sample_sizes:
    for run in range(runs_naswot):

      # total time to perform these trainings
      tempo = 0

      start = time.time()

      # list to trace the 12 epoch accuracies
      accuracies = []
      nets = np.random.randint(0,len(api),size=sample_size)

      for i in nets:
        # get_more_info gives us all the informations we need (accuracy and time to train at 12 epoches)
        accuracies.append(api.get_more_info(int(i),dataset,is_random=False,hp=12)['test-accuracy']) 
        tempo += api.get_more_info(int(i),dataset,is_random=False,hp=12)['train-all-time']

      # we take the best network for 12 epoches test accuracy
      ind_best = np.argmax(accuracies)
      
      # consider the 200 epoch test accuracy 
      final_acc = api.get_more_info(int(nets[ind_best]),dataset,is_random=False,hp=200)['test-accuracy']
      tempo += (time.time()-start)
      csv_dict = {'Dataset': dataset, 'size':sample_size, 'Network': nets[ind_best], 'Accuracy12': accuracies[ind_best], 'Accuracy200': final_acc, 'Time': tempo}
      result = pd.DataFrame([csv_dict])
      result.to_csv('naswot_accuracies.csv', mode='a', index=False, header=False )


In [ ]:
df = pd.read_csv("/content/naswot_accuracies.csv",names = ['Dataset','size','Network','Accuracy12','Accuracy200','Time'], header = None, index_col=['Dataset','Network'])

In [ ]:
accuracy12 = df.groupby(by = ['Dataset','size'])['Accuracy12'].describe()[['mean','std']]
accuracy200 = df.groupby(by = ['Dataset','size'])['Accuracy200'].describe()[['mean','std']]
time = df.groupby(by = ['Dataset','size'])['Time'].describe()[['mean','std']]

In [ ]:
data = pd.concat([accuracy12,accuracy200,time],axis=1)
data.to_csv('stats_accuracy.csv')

In [ ]:
data.columns = ['acc12_mean','acc12_std','acc_mean','acc_std','time_mean','time_std']